In [1]:
!python -m pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m pip install openpyxl



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import re

base_path = r"G:\My Drive\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
    FEBRUARY_2022_METADATA.xlsx (metadata)
  March:
    MARCH_2022.csv (survey)
    MARCH_2022_METADATA.x

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

In [5]:
# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 1)

# View the first few rows
second_sheet.head()


NameError: name 'pd' is not defined

In [ ]:
# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 0)

# View the first few rows
second_sheet.head()


,QUEST,Questionnaire,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,_IDS0,(Id Items),NaN,NaN
1,NaN,NaN,NaN,NaN,PUFREG,Region
2,NaN,NaN,NaN,NaN,PUFPRV,Province
3,NaN,NaN,NaN,NaN,PUFPRRCD,Province Recode
4,NaN,NaN,NaN,NaN,PUFHHNUM,Household Unique Sequential Number


In [ ]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [ ]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head(50)


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES
7,PUFPWGTPRV,Final Weight Based on Projection (provincial p...
8,PUFSVYMO,Survey Month
9,PUFSVYYR,Survey Year
10,PUFPSU,Psu Number
11,PUFRPL,Replicate
